In [264]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [265]:
plt.rcParams.update({"text.usetex": False,
                     "font.size": 20,
                     "pdf.fonttype": 42,
                     "ps.fonttype": 42})

In [266]:
target_latency = 160

In [267]:
# local_df = pd.read_csv("../measurements/local_long/map.csv")
# root_folder = "../measurements/real_bw_test_15fps/"

local_df = pd.read_csv("../measurements_vidvrd/local_test/map.csv")
root_folder = "../measurements_vidvrd/test_ltl_15_1/"

# local_df = pd.read_csv("../measurements_bev/local_test/map.csv")
# root_folder = "../measurements_bev/test_ltl_15_2/"

map_df = pd.read_csv(root_folder+"map.csv")
cha_df = pd.read_csv(root_folder+"characteristic.csv")
mag_df = pd.read_csv(root_folder+"manager.csv")
# mag_df = pd.read_csv("../measurements_vidvrd/test_ltl_15/manager.csv")
tim_df = pd.read_csv(root_folder+"time.csv")

cal_time = []
for i in range(len(tim_df)):
    if tim_df["technique"][i] == 2:
        cal_time.append(tim_df["compression_time"][i]/32)
    elif tim_df["technique"][i] == 3:
        cal_time.append(tim_df["compression_time"][i]-50)
    else:
        cal_time.append(tim_df["compression_time"][i])
tim_df["compression_time"] = cal_time

tim_df["transmission_time"] = cha_df["datasize_est"]*8/cha_df["consumed_bw"]*1000
tim_df["overall_est"] = tim_df["model_head_time"]*10+tim_df["model_tail_time"]+tim_df["framework_head_time"]+tim_df["framework_tail_time"] + tim_df["compression_time"]+tim_df["decompression_time"]+ tim_df["transmission_time"]

In [268]:
drops = []

for i in range(400):
    try:
        d = mag_df[mag_df["frame_id"]==i+1]["drop"].tolist()[0]
        if d >0.4:
            d=0.4
        drops.append(d)
    except:
        drops.append(drops[-1])


df = pd.DataFrame()
df["traget_drops"] = drops

In [269]:
df["map_drop"] = local_df["map"] - map_df["map"]
df["sen_drop"] = local_df["sensitivity"] - map_df["sensitivity"]

In [270]:
df["bandwidth"] = cha_df["bandwidth"]
df["bw_com"] = cha_df["consumed_bw"]
df["overall_time"] = tim_df["overall_est"]

In [271]:
df = df[200:]

In [272]:
df

,traget_drops,map_drop,sen_drop,bandwidth,bw_com,overall_time
200,0.4,0.133333,0.00,9000000.0,3.024197e+06,150.114159
201,0.4,0.066667,0.00,9000000.0,3.434951e+06,150.934063
202,0.4,0.000000,0.00,9000000.0,3.422754e+06,150.631914
203,0.4,0.000000,0.00,9000000.0,2.202689e+06,150.183274
204,0.4,-0.125000,-0.25,9000000.0,1.680787e+06,149.757710
...,...,...,...,...,...,...
395,0.4,0.500000,0.50,9000000.0,3.274623e+06,151.524685
396,0.4,0.000000,0.00,9000000.0,1.927082e+06,150.830061
397,0.4,0.000000,0.00,9000000.0,3.755410e+06,151.146767
398,0.4,0.000000,0.00,9000000.0,1.943803e+06,150.816742


In [273]:
good_df = df[df["map_drop"]<df["traget_drops"]]
good_df= good_df[good_df["sen_drop"]<good_df["traget_drops"]]
good_df= good_df[good_df["bw_com"]<good_df["bandwidth"]]
good_df= good_df[good_df["overall_time"]<target_latency]

In [274]:
good_df

,traget_drops,map_drop,sen_drop,bandwidth,bw_com,overall_time
200,0.4,0.133333,0.00,9000000.0,3.024197e+06,150.114159
201,0.4,0.066667,0.00,9000000.0,3.434951e+06,150.934063
202,0.4,0.000000,0.00,9000000.0,3.422754e+06,150.631914
203,0.4,0.000000,0.00,9000000.0,2.202689e+06,150.183274
204,0.4,-0.125000,-0.25,9000000.0,1.680787e+06,149.757710
...,...,...,...,...,...,...
394,0.4,0.000000,0.00,9000000.0,1.792328e+06,150.836109
396,0.4,0.000000,0.00,9000000.0,1.927082e+06,150.830061
397,0.4,0.000000,0.00,9000000.0,3.755410e+06,151.146767
398,0.4,0.000000,0.00,9000000.0,1.943803e+06,150.816742


In [275]:
len(good_df)/200

0.915